# IMPORTS

In [1]:
import json
import os
import time
from AxonDeepSeg.config import *

## Setting hyperparameters and creating the file _config_

Quick explanations

In [2]:
# Description du fichier config :
# network_learning_rate : float : No idea, but certainly linked to the back propagation ? Default : 0.0005.
# network_n_classes : int : number of labels in the output. Default : 2.
# dropout : float : between 0 and 1 : percentage of neurons we want to keep. Default : 0.75.
# network_depth : int : number of layers WARNING : factualy, there will be 2*network_depth layers. Default : 6.
# network_convolution_per_layer : list of int, length = network_depth : number of convolution per layer. Default : [1 for i in range(network_depth)].
# network_size_of_convolutions_per_layer : list of lists of int [number of layers[number_of_convolutions_per_layer]] : Describe the size of each convolution filter.
# Default : [[3 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)].

# network_features_per_convolution : list of lists of int [number of layers[number_of_convolutions_per_layer[2]] : Numer of different filters that are going to be used.
# Default : [[[64,64] for k in range(network_convolution_per_layer[i])] for i in range(network_depth)]. WARNING ! network_features_per_convolution[k][1] = network_features_per_convolution[k+1][0].

# network_trainingset : string : String describing the dataset used for the training.

Name of the .json file where the network configuration will be stored

In [3]:
filename = '/config_network.json'

## Setting the main parameters of the network

If multiple training sets are available, you can select which one you want to use here.

In [4]:
trainingset_name = 'SEM_2classes_reduced'

We then set the main parameters and the architecture of the network

In [5]:
network_learning_rate = 0.001
network_n_classes = 2
dropout = 0.75
network_depth = 4
network_convolution_per_layer = [3 for i in range(network_depth)]
network_size_of_convolutions_per_layer = [[5,5,5],[3,3,3],[3,3,3],[3,3,3]]#[[3 for k in range(network_convolution_per_layer[i])] for i in range(network_depth)]
network_features_per_convolution = [[[1,10],[10,20],[20,30]],[[30,40],[40,50],[50,60]],[[60,70],[70,80],[80,90]],[[90,100],[100,110],[110,120]],
                                    ]
network_batch_size = 8
network_batch_norm_decay = 0.90
network_batch_norm = True
downsampling = 'convolution'


# Note: type can be all (we will do all transformations set to True (default) on each patch) or random (we will choose one random transformation
# (among the ones set to True) and apply each to the current patch)
# You can also set 'type' to 'none' (string), then we won't do any transformation.

network_data_augmentation = {'type':'all', 
                                      'transformations':{'shifting':False, 'rescaling':False,
                                                         'random_rotation':True, 'elastic':False, 'flipping':False,
                                                         'noise_addition':False}
                            }



thresholds = [0,0.5]

weighted_cost = False

config = {
    'network_n_classes': network_n_classes,
    'network_thresholds': thresholds,
    'network_learning_rate': network_learning_rate,
    'network_batch_size': network_batch_size,
    'network_dropout': dropout,
    'network_depth': network_depth,
    'network_batch_norm_decay': network_batch_norm_decay,
    'network_convolution_per_layer': network_convolution_per_layer,
    'network_size_of_convolutions_per_layer': network_size_of_convolutions_per_layer,
    'network_features_per_convolution': network_features_per_convolution,
    'network_trainingset': trainingset_name,
    'network_downsampling': downsampling,
    'network_weighted_cost': weighted_cost,
    'network_batch_norm': network_batch_norm,
    'network_data_augmentation': network_data_augmentation
}




## Preparing the training phase

In [6]:
path_training = '../data/'+trainingset_name+'/training/'

## Actually training the model

By default, the name is the current date. You can uncomment the second line dir_name if you want to give a specific name to the folder, but beware: it will erase the contents of the folder if it already exists !

The model data will be stored in a folder with this name, as well as the summaries that are going to be readable on tensorboard.

In [7]:
dir_name = time.strftime("%Y-%m-%d") + '_' + time.strftime("%H-%M-%S") 
#dir_name = 'test_bn'
path_model = os.path.join('../models/', dir_name)

if not os.path.exists(path_model):
    os.makedirs(path_model)
    
path_model_init = None 


# Specify here the path to the initial model if needed.

if os.path.exists(path_model+filename): # if there is already a configfile
    with open(path_model+filename, 'r') as fd:
        config_network = json.loads(fd.read())
else: # There is no config file for the moment
    with open(path_model+filename, 'w') as f:
        json.dump(config, f, indent=2)
    with open(path_model+filename, 'r') as fd:
        config_network = json.loads(fd.read())
        
        
# Ensure that the config file has all the necessary parameters

config_network = update_config(default_configuration(), config_network)
    
################################################### Training phase ###################################################

# Note : possible values for gpu: None, '/gpu:0'. If used withing CUDA_VISIBLE_DEVICES=?, then it will use gpu number ? by default
from AxonDeepSeg.train_network import train_model
train_model(path_training, path_model, config_network,path_model_init=path_model_init,gpu='/gpu:0')

('Layer: ', 0, ' Conv: ', 0, 'Features: ', [1, 10])
('Size:', 5)
('Layer: ', 0, ' Conv: ', 1, 'Features: ', [10, 20])
('Size:', 5)
('Layer: ', 0, ' Conv: ', 2, 'Features: ', [20, 30])
('Size:', 5)
('Layer: ', 1, ' Conv: ', 0, 'Features: ', [30, 40])
('Size:', 3)
('Layer: ', 1, ' Conv: ', 1, 'Features: ', [40, 50])
('Size:', 3)
('Layer: ', 1, ' Conv: ', 2, 'Features: ', [50, 60])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 0, 'Features: ', [60, 70])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 1, 'Features: ', [70, 80])
('Size:', 3)
('Layer: ', 2, ' Conv: ', 2, 'Features: ', [80, 90])
('Size:', 3)
('Layer: ', 3, ' Conv: ', 0, 'Features: ', [90, 100])
('Size:', 3)
('Layer: ', 3, ' Conv: ', 1, 'Features: ', [100, 110])
('Size:', 3)
('Layer: ', 3, ' Conv: ', 2, 'Features: ', [110, 120])
('Size:', 3)
('Layer: ', 0, ' Conv: ', 0, 'Features: ', [1, 10])
('Size:', 5)
('Layer: ', 0, ' Conv: ', 1, 'Features: ', [10, 20])
('Size:', 5)
('Layer: ', 0, ' Conv: ', 2, 'Features: ', [20, 30])
('Size:', 5)
('Layer

/home/GRAMES.POLYMTL.CA/mawab/ads/tf11-ads/local/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "




----Scores on validation:---
----
 Last epoch: 1
 Accuracy: 0.894121;
 Loss: 0.26428;


----Scores on validation:---
----
 Last epoch: 2
 Accuracy: 0.924121;
 Loss: 0.205835;


----Scores on validation:---
----
 Last epoch: 3
 Accuracy: 0.91899;
 Loss: 0.219386;


----Scores on validation:---
----
 Last epoch: 4
 Accuracy: 0.91861;
 Loss: 0.211732;


----Scores on validation:---
----
 Last epoch: 5
 Accuracy: 0.929625;
 Loss: 0.188525;


----Scores on validation:---
----
 Last epoch: 6
 Accuracy: 0.912386;
 Loss: 0.226854;
Model saved in file: ../models/2017-06-21_18-47-56/model.ckpt


----Scores on validation:---
----
 Last epoch: 7
 Accuracy: 0.930973;
 Loss: 0.183012;


----Scores on validation:---
----
 Last epoch: 8
 Accuracy: 0.933888;
 Loss: 0.180461;


----Scores on validation:---
----
 Last epoch: 9
 Accuracy: 0.930077;
 Loss: 0.185205;


----Scores on validation:---
----
 Last epoch: 10
 Accuracy: 0.936624;
 Loss: 0.172208;


----Scores on validation:---
----
 Last epoch: 1

Model saved in file: ../models/2017-06-21_18-47-56/model.ckpt


----Scores on validation:---
----
 Last epoch: 85
 Accuracy: 0.936181;
 Loss: 0.165052;


----Scores on validation:---
----
 Last epoch: 86
 Accuracy: 0.935922;
 Loss: 0.164577;


----Scores on validation:---
----
 Last epoch: 87
 Accuracy: 0.936825;
 Loss: 0.163927;


----Scores on validation:---
----
 Last epoch: 88
 Accuracy: 0.934974;
 Loss: 0.16825;


----Scores on validation:---
----
 Last epoch: 89
 Accuracy: 0.936173;
 Loss: 0.16717;


----Scores on validation:---
----
 Last epoch: 90
 Accuracy: 0.93763;
 Loss: 0.161687;


----Scores on validation:---
----
 Last epoch: 91
 Accuracy: 0.933382;
 Loss: 0.17185;
Model saved in file: ../models/2017-06-21_18-47-56/model.ckpt


----Scores on validation:---
----
 Last epoch: 92
 Accuracy: 0.935151;
 Loss: 0.166849;


----Scores on validation:---
----
 Last epoch: 93
 Accuracy: 0.935657;
 Loss: 0.167177;


----Scores on validation:---
----
 Last epoch: 94
 Accuracy: 0.93662



----Scores on validation:---
----
 Last epoch: 167
 Accuracy: 0.940033;
 Loss: 0.160237;


----Scores on validation:---
----
 Last epoch: 168
 Accuracy: 0.93735;
 Loss: 0.166848;


----Scores on validation:---
----
 Last epoch: 169
 Accuracy: 0.937783;
 Loss: 0.159702;
Model saved in file: ../models/2017-06-21_18-47-56/model.ckpt


----Scores on validation:---
----
 Last epoch: 170
 Accuracy: 0.937509;
 Loss: 0.161647;


----Scores on validation:---
----
 Last epoch: 171
 Accuracy: 0.935766;
 Loss: 0.169761;


----Scores on validation:---
----
 Last epoch: 172
 Accuracy: 0.938438;
 Loss: 0.163001;


----Scores on validation:---
----
 Last epoch: 173
 Accuracy: 0.937395;
 Loss: 0.16469;


----Scores on validation:---
----
 Last epoch: 174
 Accuracy: 0.937572;
 Loss: 0.162892;


----Scores on validation:---
----
 Last epoch: 175
 Accuracy: 0.939483;
 Loss: 0.159494;


----Scores on validation:---
----
 Last epoch: 176
 Accuracy: 0.939559;
 Loss: 0.161678;
Model saved in file: ../models

...



----Scores on validation:---
----
 Last epoch: 332
 Accuracy: 0.937527;
 Loss: 0.165077;
Model saved in file: ../models/2017-06-21_18-47-56/model.ckpt


----Scores on validation:---
----
 Last epoch: 333
 Accuracy: 0.938607;
 Loss: 0.164224;


----Scores on validation:---
----
 Last epoch: 334
 Accuracy: 0.938952;
 Loss: 0.163396;


----Scores on validation:---
----
 Last epoch: 335
 Accuracy: 0.939089;
 Loss: 0.170959;


----Scores on validation:---
----
 Last epoch: 336
 Accuracy: 0.938377;
 Loss: 0.169327;


----Scores on validation:---
----
 Last epoch: 337
 Accuracy: 0.9402;
 Loss: 0.157794;


----Scores on validation:---
----
 Last epoch: 338
 Accuracy: 0.938371;
 Loss: 0.16851;


----Scores on validation:---
----
 Last epoch: 339
 Accuracy: 0.937749;
 Loss: 0.165605;
Model saved in file: ../models/2017-06-21_18-47-56/model.ckpt


----Scores on validation:---
----
 Last epoch: 340
 Accuracy: 0.939822;
 Loss: 0.159466;


----Scores on validation:---
----
 Last epoch: 341
 Accura



----Scores on validation:---
----
 Last epoch: 414
 Accuracy: 0.939972;
 Loss: 0.161152;


----Scores on validation:---
----
 Last epoch: 415
 Accuracy: 0.940144;
 Loss: 0.16027;


----Scores on validation:---
----
 Last epoch: 416
 Accuracy: 0.939796;
 Loss: 0.16415;


----Scores on validation:---
----
 Last epoch: 417
 Accuracy: 0.939621;
 Loss: 0.165533;
Model saved in file: ../models/2017-06-21_18-47-56/model.ckpt


----Scores on validation:---
----
 Last epoch: 418
 Accuracy: 0.939554;
 Loss: 0.170483;


----Scores on validation:---
----
 Last epoch: 419
 Accuracy: 0.939484;
 Loss: 0.16455;


----Scores on validation:---
----
 Last epoch: 420
 Accuracy: 0.941031;
 Loss: 0.160386;


----Scores on validation:---
----
 Last epoch: 421
 Accuracy: 0.940427;
 Loss: 0.160364;


----Scores on validation:---
----
 Last epoch: 422
 Accuracy: 0.941721;
 Loss: 0.155968;


----Scores on validation:---
----
 Last epoch: 423
 Accuracy: 0.941772;
 Loss: 0.158068;
Model saved in file: ../models/



----Scores on validation:---
----
 Last epoch: 496
 Accuracy: 0.940525;
 Loss: 0.164527;


----Scores on validation:---
----
 Last epoch: 497
 Accuracy: 0.939182;
 Loss: 0.165506;


----Scores on validation:---
----
 Last epoch: 498
 Accuracy: 0.940315;
 Loss: 0.160491;


----Scores on validation:---
----
 Last epoch: 499
 Accuracy: 0.941243;
 Loss: 0.159212;


----Scores on validation:---
----
 Last epoch: 500
 Accuracy: 0.940021;
 Loss: 0.166608;


----Scores on validation:---
----
 Last epoch: 501
 Accuracy: 0.940308;
 Loss: 0.165113;


----Scores on validation:---
----
 Last epoch: 502
 Accuracy: 0.941895;
 Loss: 0.153793;
Model saved in file: ../models/2017-06-21_18-47-56/model.ckpt


----Scores on validation:---
----
 Last epoch: 503
 Accuracy: 0.93873;
 Loss: 0.165727;


----Scores on validation:---
----
 Last epoch: 504
 Accuracy: 0.940539;
 Loss: 0.163725;


----Scores on validation:---
----
 Last epoch: 505
 Accuracy: 0.939181;
 Loss: 0.16288;


----Scores on validation:---



----Scores on validation:---
----
 Last epoch: 579
 Accuracy: 0.93859;
 Loss: 0.165317;


----Scores on validation:---
----
 Last epoch: 580
 Accuracy: 0.940994;
 Loss: 0.159038;
Model saved in file: ../models/2017-06-21_18-47-56/model.ckpt


----Scores on validation:---
----
 Last epoch: 581
 Accuracy: 0.939646;
 Loss: 0.164171;


----Scores on validation:---
----
 Last epoch: 582
 Accuracy: 0.941646;
 Loss: 0.160961;


----Scores on validation:---
----
 Last epoch: 583
 Accuracy: 0.940407;
 Loss: 0.159137;


----Scores on validation:---
----
 Last epoch: 584
 Accuracy: 0.938966;
 Loss: 0.163955;


----Scores on validation:---
----
 Last epoch: 585
 Accuracy: 0.940473;
 Loss: 0.159006;


----Scores on validation:---
----
 Last epoch: 586
 Accuracy: 0.941572;
 Loss: 0.157703;
Model saved in file: ../models/2017-06-21_18-47-56/model.ckpt


----Scores on validation:---
----
 Last epoch: 587
 Accuracy: 0.941125;
 Loss: 0.156393;


----Scores on validation:---
----
 Last epoch: 588
 Accu

KeyboardInterrupt: 